In [32]:
import numpy as np
import chaospy as cp
import GPy

class SequentialPCKriging:
    def __init__(self, input_dist, order=3):
        self.input_dist = input_dist
        self.expansion = cp.generate_expansion(order, input_dist)
        self.gp_model = None

    def fit_pce(self, x, y):
        if x.ndim == 2 and x.shape[1] == 1:
            x = x.flatten()
        if y.ndim == 2 and y.shape[1] == 1:
            y = y.flatten()
        self.pce_model = cp.fit_regression(self.expansion, x, y)

    def fit_kriging(self, x, y):
        discrepancy = y - self.pce_model(x)
        kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
        self.gp_model = GPy.models.GPRegression(x, discrepancy, kernel)
        self.gp_model.optimize()

    def fit(self, x, y):
        self.fit_pce(x, y)
        self.fit_kriging(x, y)

    def predict(self, x_new):
        pce_pred = self.pce_model(x_new)
        gp_pred, gp_var = self.gp_model.predict(x_new)
        return pce_pred + gp_pred, np.sqrt(gp_var)

In [33]:
from scipy.stats import norm

def expected_improvement(X, X_sample, Y_sample, gp, xi=0.01):
    '''
    Computes the EI at points X based on existing samples X_sample
    and Y_sample using a Gaussian process surrogate model.

    Args:
    X: Points at which EI shall be computed (m x d).
    X_sample: Sample locations (n x d).
    Y_sample: Sample values (n x 1).
    gp: A Gaussian process fitted to samples.
    xi: Exploitation-exploration trade-off parameter.

    Returns:
    Expected improvements at points X.
    '''
    # Making a prediction at point X
    mu, sigma = gp.predict(X)

    # Calculating the improvement
    mu_sample_opt = np.max(Y_sample)
    imp = mu - mu_sample_opt - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    ei[sigma == 0.0] = 0.0  # If sigma is 0, the expected improvement is 0

    return ei


In [34]:
from scipy.optimize import minimize

def propose_location(acquisition, X_sample, Y_sample, kriging_model, bounds, n_restarts=25):
    """
    Proposes the next sampling point by optimizing the acquisition function.

    Args:
        acquisition: Acquisition function.
        X_sample: Sample locations (n x d).
        Y_sample: Sample values (n x 1).
        kriging_model: A Gaussian process model.
        bounds: Bounds of the design space.
        n_restarts: Number of restarts for the optimizer.

    Returns:
        Location of the next sampling point.
    """
    dim = X_sample.shape[1]
    min_val = 1
    min_x = None

    def min_obj(X):
        # Minimization objective is the negative acquisition function
        return -acquisition(X.reshape(-1, dim), X_sample, Y_sample, kriging_model)

    # Start with n_restart different random choices for the starting point
    for x0 in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, dim)):
        res = minimize(min_obj, x0=x0, bounds=bounds, method='L-BFGS-B')
        if res.fun < min_val:
            min_val = res.fun
            min_x = res.x

    return min_x.reshape(-1, 1)


In [35]:
from scipy.optimize import minimize

def optimize(f, bounds, input_dist,  n_iter=25, initial_samples=5, xi=0.01):
    """
    Main optimization loop.

    Args:
        f: Objective function.
        bounds: Bounds of the design space (2d array).
        n_iter: Number of iterations.
        initial_samples: Number of initial random samples.
        xi: Exploration-exploitation trade-off parameter.

    Returns:
        Optimal value and corresponding point.
    """
    # Randomly sample the initial design space
    X_sample = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(initial_samples, len(bounds)))
    Y_sample = f(X_sample)

    # Initialize Kriging model
    kriging_model = SequentialPCKriging(input_dist, order=5)

    kriging_model.fit(X_sample, Y_sample)

    for i in range(n_iter):
        # Update the model
        kriging_model.fit(X_sample, Y_sample)
        # Obtain next sampling point from the acquisition function (expected_improvement)
        X_next = propose_location(expected_improvement, X_sample, Y_sample, kriging_model, bounds=bounds)

        # Obtain next noisy sample from the objective function
        Y_next = f(X_next)

        # Add the new sample point to the existing set of samples
        X_sample = np.vstack((X_sample, X_next))
        Y_sample = np.vstack((Y_sample, Y_next))

    return np.max(Y_sample), X_sample[np.argmax(Y_sample)]


In [36]:
def f_simple_test(x):
    return -1 * (x - 2)**2 + 5

# Test the optimization function
bounds = np.array([[0, 4]])  # Bounds of the design space
opt_val, opt_point = optimize(f_simple_test, bounds, cp.Uniform(0,4), n_iter=25, initial_samples=5, xi=0.01)

print("Optimal Value:", opt_val)
print("Optimal Point:", opt_point)

Optimal Value: 5.0
Optimal Point: [2.]
